In [1]:
from glob import glob
from pymatgen.core import Structure, Element, Molecule
import pymatgen.io.feff
import numpy as np
from scipy.spatial.distance import cdist
from pymatgen.vis.structure_vtk import StructureVis
import nglview
from PyAstronomy import pyasl


In [2]:
names=glob("data/*.xyz")
structure=Molecule.from_file(names[2])
structure

Molecule Summary
Site: Pt (-4.1580, 4.1580, -5.1620)
Site: Pt (-4.1580, 1.3860, -5.1620)
Site: Pt (-4.1580, -1.3860, -5.1620)
Site: Pt (-1.3860, 4.1580, -5.1620)
Site: Pt (-1.3860, 1.3860, -5.1620)
Site: Pt (1.3860, 4.1580, -5.1620)
Site: Pt (-4.1580, -4.1580, -5.1620)
Site: Pt (-1.3860, -1.3860, -5.1620)
Site: Pt (-1.3860, -4.1580, -5.1620)
Site: Pt (1.3860, 1.3860, -5.1620)
Site: Pt (1.3860, -1.3860, -5.1620)
Site: Pt (4.1580, 4.1580, -5.1620)
Site: Pt (4.1580, 1.3860, -5.1620)
Site: Pt (1.3860, -4.1580, -5.1620)
Site: Pt (4.1580, -1.3860, -5.1620)
Site: Pt (4.1580, -4.1580, -5.1620)
Site: Pt (-5.5440, 2.7720, -3.2020)
Site: Pt (-6.9300, 1.3860, -1.2420)
Site: Pt (-2.7720, 5.5440, -3.2020)
Site: Pt (-4.1580, 4.1580, -1.2420)
Site: Pt (-1.3860, 6.9300, -1.2420)
Site: Pt (-5.5440, -2.7720, -3.2020)
Site: Pt (-6.9300, -1.3860, -1.2420)
Site: Pt (-5.5440, 0.0000, -3.2020)
Site: Pt (-2.7720, 0.0000, -3.2020)
Site: Pt (-4.1580, 1.3860, -1.2420)
Site: Pt (-4.1580, -1.3860, -1.2420)
Site: Pt

In [3]:
absorber_species=Element("Pt")
absorber_list=np.where(np.array(structure.species)==absorber_species)[0]

In [4]:
nglview.show_pymatgen(structure)


NGLWidget()

In [5]:
pot=pymatgen.io.feff.inputs.Potential(structure,int(1))
pot.pot_dict


{'Pt': 1}

In [6]:
neighbor_list(structure,2,ipot_dist=4.5)



NameError: name 'neighbor_list' is not defined

In [180]:
from tabulate import tabulate
def neighbor_list(structure,absorber_index,ipot_dist=4.5,cluster_size=10):
    clusters=pymatgen.io.feff.inputs.Atoms(structure, int(absorber_index), cluster_size).get_lines()
    cluster_list,ele_list=cluster_to_numpy(clusters)
    neighbors=np.where(cdist([cluster_list[0]],cluster_list)[0]<ipot_dist)[0]
    neighbors=np.delete(neighbors,np.where(neighbors==0))
    neighbor_dict=dict()
    for i in range(len(neighbors)):
        key=len(np.where(cdist([cluster_list[i]],cluster_list)[0]<ipot_dist)[0])
        if key not in neighbor_dict:
            neighbor_dict[key]={ele_list[i]:[neighbors[i]]}
        else:
            if ele_list[i] not in list(neighbor_dict[key].keys()):
                neighbor_dict[key][ele_list[i]]=[neighbors[i]]
            else:
                neighbor_dict[key][ele_list[i]].append(neighbors[i])
    neighbor_rewrite=dict()

    for key in list(neighbor_dict.keys()):
        for species in list(neighbor_dict[key].keys()):
            if species not in list(neighbor_rewrite.keys()):
                neighbor_rewrite[species]=[np.array(neighbor_dict[key][species])]
            else:
                neighbor_rewrite[species].append(np.array(neighbor_dict[key][species]))
        
    return neighbor_rewrite


def calc_pot_atoms_list(path, absorber = None, absorber_list = [], ipot_dist = 4.5, cluster_size = 10):
    """
    Calculate the POTENTIAL and ATOMS card of feff input of given structure.
    """
    if path.split('.')[1]=='xyz':
        structure=Molecule.from_file(path)
    else:
        structure = Structure.from_file(path)
    
    pot_atoms_list = []
    
    if len(absorber_list) == 0:
        if absorber is None:
            raise ValueError("Please specify the absorber element.")
        
    
    for ii in range(len(absorber_list)):
            clusters=np.array(pymatgen.io.feff.inputs.Atoms(structure, int(absorber_list[ii]), cluster_size).get_lines())
            clusters = clusters[np.argsort(clusters[:, 5].astype(float))]
            cluster_list,ele_list=cluster_to_numpy(clusters)
            neighbor_dict=neighbor_list(structure,absorber_list[ii],ipot_dist,cluster_size)
            
            elements=structure.elements
            num_ele_total=len(elements)
            species=structure.species
            central_element = species[absorber_list[ii]]
            ipotrow = [[0,central_element.Z,central_element.symbol,-1,-1,1,0]]
            pot_num=1
            # pot_dict=dict()
            num_atom=0
            for i in range(num_ele_total):
                num_ele_shell=0
                ele1=np.where(np.array(ele_list)==elements[i])[0]
                ele1=np.delete(ele1,np.where(ele1==0))
                num_ele=len(np.where(np.array(ele_list)==elements[i])[0])-1#number of one kind of element
                atom_list=[]
                for k in range(len(neighbor_dict[elements[i]])):
                    ipotrow.append([pot_num,elements[i].Z,elements[i].symbol,-1,-1,len(neighbor_dict[elements[i]][k]),0])
                    for j in range(len(neighbor_dict[elements[i]][k])):
                        clusters[neighbor_dict[elements[i]][k][j]][3]=str(pot_num)
                        atom_list.append(neighbor_dict[elements[i]][k][j])
                    # pot_dict[neighbor_dict[elements[i]][k][j]]=pot_num
                    num_ele_shell+=len(neighbor_dict[elements[i]][k])
                    pot_num+=1
                num_atom+=num_ele_shell
                residual=[item for item in range(len(ele1)) if item not in atom_list]
                if num_ele-num_ele_shell>0:
                    ipotrow.append([pot_num,elements[i].Z,elements[i].symbol,-1,-1,num_ele-num_ele_shell,0])
                    for j in range(len(residual)):
                        clusters[num_ele_shell+j-1][3]=str(pot_num)
                        # pot_dict[num_atom+j]=pot_num
                    pot_num+=1
            unique_potential=np.unique(clusters[:,3])
            map_potential = {unique_potential[i]: str(i) for i in range(len(unique_potential))}
            pot_index=list(np.array(ipotrow)[:,0]) 
            if len(map_potential)!=len(ipotrow):
                    miss_pot=set(pot_index).difference(list(map_potential.keys()))
                    raise ValueError(f"The radius is too short to include all potentials, please choose a larger radius(missing potential {miss_pot}).")
            pot_atoms_list.append({"potential": tabulate(ipotrow, tablefmt="plain"), "atoms": tabulate(cluster, tablefmt="plain")})
        
    return pot_atoms_list
    
def cluster_to_numpy(clusters):
    cluster_list=[]
    ele=[]
    for cluster in clusters:
        cluster_list.append([np.float32(cluster[0]),np.float32(cluster[1]),np.float32(cluster[2])])
        ele.append(Element(cluster[4]))
    return np.array(cluster_list),ele




In [181]:
neighbor_list(structure,2,ipot_dist=4.5,cluster_size=10)

{Element Pt: [array([1, 2, 6], dtype=int64),
  array([3], dtype=int64),
  array([4, 9], dtype=int64),
  array([5], dtype=int64),
  array([7], dtype=int64),
  array([ 8, 10], dtype=int64)]}

In [182]:
from pprint import pprint
pprint(calc_pot_atoms_list(names[2], absorber = "Pt", absorber_list = [1,2,3], ipot_dist = 4.5, cluster_size = 10))

[{'atoms': ' 0       0      0     0  Pt  0         0\n'
           '-1.386  -1.386  1.96  1  Pt  2.77193  21\n'
           '-1.386   1.386  1.96  1  Pt  2.77193  23\n'
           ' 1.386   1.386  1.96  1  Pt  2.77193  24\n'
           ' 1.386  -1.386  1.96  1  Pt  2.77193  36\n'
           ' 0       2.772  0     1  Pt  2.772     2\n'
           ' 0      -2.772  0     1  Pt  2.772     6\n'
           ' 2.772   0      0     1  Pt  2.772     7\n'
           ' 0       0      3.92  1  Pt  3.92     26\n'
           ' 2.772   2.772  0     1  Pt  3.9202    4\n'
           ' 2.772  -2.772  0     1  Pt  3.9202    8\n'
           ' 2.772   0      3.92  1  Pt  4.80108  38\n'
           ' 0      -2.772  3.92  1  Pt  4.80108  35\n'
           '-2.772   0      3.92  1  Pt  4.80108  22\n'
           ' 0       2.772  3.92  1  Pt  4.80108  25\n'
           '-1.386   4.158  1.96  1  Pt  4.8012   16\n'
           ' 1.386   4.158  1.96  1  Pt  4.8012   27\n'
           ' 1.386  -4.158  1.96  1  Pt  4.8012 

In [98]:
clusters=np.array(pymatgen.io.feff.inputs.Atoms(structure, int(2), 10).get_lines())
cluster = clusters[np.argsort(clusters[:, 5].astype(float))]
cluster

array([['0.000000', '0.000000', '0.000000', '0', 'Pt', '0.0', '0'],
       ['-1.386000', '-1.386000', '1.960000', '1', 'Pt', '2.771929',
        '21'],
       ['-1.386000', '1.386000', '1.960000', '1', 'Pt', '2.771929', '23'],
       ['1.386000', '1.386000', '1.960000', '1', 'Pt', '2.771929', '24'],
       ['1.386000', '-1.386000', '1.960000', '1', 'Pt', '2.771929', '36'],
       ['0.000000', '2.772000', '0.000000', '1', 'Pt', '2.772000', '2'],
       ['0.000000', '-2.772000', '0.000000', '1', 'Pt', '2.772000', '6'],
       ['2.772000', '0.000000', '0.000000', '1', 'Pt', '2.772000', '7'],
       ['0.000000', '0.000000', '3.920000', '1', 'Pt', '3.920000', '26'],
       ['2.772000', '2.772000', '0.000000', '1', 'Pt', '3.920200', '4'],
       ['2.772000', '-2.772000', '0.000000', '1', 'Pt', '3.920200', '8'],
       ['2.772000', '0.000000', '3.920000', '1', 'Pt', '4.801082', '38'],
       ['0.000000', '-2.772000', '3.920000', '1', 'Pt', '4.801082', '35'],
       ['-2.772000', '0.000000', '